In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\End_to_End_MLProject_with-MLFLOW\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\End_to_End_MLProject_with-MLFLOW'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from src.MLProject_With_MLFLOW.constants import *
from src.MLProject_With_MLFLOW.utils.common import read_yaml,create_directory

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
    
        create_directory([self.config.artifacts_root])


    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directory([config.root_dir])


        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from src.MLProject_With_MLFLOW.utils.common import logger
from src.MLProject_With_MLFLOW.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config

    def download_file(self):
        """this method is responsible for data downloading"""

        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} is downloaded with following info: \n{header}")

        else:
            logger.info(f"file is already exist {get_size(Path(self.config.local_data_file))}")

    
    def extract_zipfile(self):
        """ this mwthod is responsible to extract data from zip file"""

        unzip_data = self.config.unzip_dir
        os.makedirs(unzip_data, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_data)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()

except Exception as e:
    raise e

[2023-10-30 15:42:12,028 - INFO - common - yaml_file config\config.yaml loaded successfully]
[2023-10-30 15:42:12,029 - INFO - common - yaml_file params.yaml loaded successfully]
[2023-10-30 15:42:12,036 - INFO - common - yaml_file schema.yaml loaded successfully]
[2023-10-30 15:42:12,041 - INFO - common - created directory atartifacts]
[2023-10-30 15:42:12,041 - INFO - common - created directory atartifacts/data_ingestion]


[2023-10-30 15:42:14,666 - INFO - 3130220649 - artifacts/data_ingestion/data.zip is downloaded with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F852:199EA4:30A18:54107:653F8173
Accept-Ranges: bytes
Date: Mon, 30 Oct 2023 10:12:06 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100021-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1698660726.243135,VS0,VE280
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 4f7361f4622df3533132d99788c6d993a8565b29
Expires: Mon, 30 Oct 2023 10:17:06 GMT
Source-Age: 0

]
